# Homework 2: Function approximation

**Your task**

- Implement experience replay for CartPole with linear function approximation.  (4p)

- **Bonus**: implement DQN with experience replay. Use a neural network instead of a linear approximator (2p):

- *Bonus points*: 1 bug found in my code (hopefully none, but it happens) = 1 extra point.
    - Bug 1: td_target calculation should use `np.max(estimator.predict(new_state))` instead of `np.amax()`
    - Bug 2: set `state = new_state` at the end of `while not done:` loop

You can borrow inspiration from tutorial 5 (`05 fa_cartpole_sklearn.py`).

In [3]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from collections import deque
from sklearn.linear_model import SGDRegressor

In [39]:
class FunctionEstimator:
    def __init__(self,env, alpha=0.001):
        self.alpha = alpha
        self.env = env
        self.n_actions = env.action_space.n
        self.possible_actions = range(env.action_space.n)
        self.initial_state = self.featurize(env.reset())
        self.state_dim = len(env.reset())
        self.w = np.ones(shape=(self.state_dim, self.n_actions), dtype=float) #parameters of the function approximating Q table
    
    def featurize(self,state):
        """
        Turn state into feature vector so it can be inputted into the function approximating Q table.
        """
        return np.tanh(state)
    
    def gradient(self, state, action):
        """
        f(state,action) = state*w_action
        df(state, action)/dw_action = state
        """
        return state
    
    def update(self, state, action, td_target):
        """
        Update the parameters by gradient descend.
        Move the result of f(state, action) closer to target = minimize MSE.
        """
        state = self.featurize(state) 
        
        target = td_target
        prediction = self.predict(state)[action]
        error = target - prediction
        gradient = self.gradient(state, action)
        
        self.w[:,action] -= self.alpha * error * (-1 * gradient)
        
    def batch_update(self, history, batch_size=5):
        history = np.array(history)
        length = history.shape[0]
        
        if length <= batch_size:
            batch = history
        else:
            idxs = np.random.choice(length, batch_size, replace=False) #select indexes of points from history
            batch = history[idxs] # select points from history to learn from
        
        for i in range(batch.shape[0]):
            [state, action, reward, new_state, td_target] = batch[i]
            self.update(state, action, td_target)
        
    def predict(self,state):
        """
        Predict Q-values of every action in a given state.
        Policy will then select which action to take based on these Q-values.
        """
        state = self.featurize(state)
        return np.dot(state, self.w).flatten() # linear function

        
def make_policy(estimator, epsilon):
    """
    Epsilon greedy policy choosing which action to take.
    """
    def policy_fn(state):
        preds = estimator.predict(state)
        if np.random.rand()>epsilon:
            action = np.argmax(preds)
        else:
            action = np.random.choice(estimator.possible_actions)
        return action
    
    return policy_fn


def run_episodes(n_episodes = 1000, gamma = 1):
    env = gym.make('CartPole-v0')

    estimator = FunctionEstimator(env)
    history = []
    episode_rewards = []

    for ep in range(n_episodes):
        state = env.reset()
        done = False
        policy = make_policy(estimator, 0.99**ep) # reduce epsilon
        ep_reward = 0

        while not done:
            action = policy(state)
            new_state, reward, done, _ = env.step(action)
            ep_reward += reward

            # Update the Q-function
            td_target = reward
            if not done:
                td_target = reward + gamma*np.max(estimator.predict(new_state))
            
            # Keep track of the states    
            history.append([state, action, reward, new_state, td_target])
            
            estimator.update(state,action,td_target)
            #estimator.batch_update(history)
            
            state = new_state

        episode_rewards.append(ep_reward)
        # Show stats
        if (ep) % 100 == 0:
            #print('*'*100)
            print("INFO: Reward at episode {} is {}, avg in last 100: {}".format(ep,ep_reward, np.average(episode_rewards[-100:])))
            
    env.close()

In [43]:
run_episodes(n_episodes=2000)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO: Reward at episode 0 is 26.0, avg in last 100: 26.0
INFO: Reward at episode 100 is 64.0, avg in last 100: 44.51
INFO: Reward at episode 200 is 69.0, avg in last 100: 80.07
INFO: Reward at episode 300 is 68.0, avg in last 100: 96.67
INFO: Reward at episode 400 is 59.0, avg in last 100: 93.01
INFO: Reward at episode 500 is 126.0, avg in last 100: 88.01
INFO: Reward at episode 600 is 67.0, avg in last 100: 95.2
INFO: Reward at episode 700 is 104.0, avg in last 100: 88.85
INFO: Reward at episode 800 is 52.0, avg in last 100: 87.56
INFO: Reward at episode 900 is 46.0, avg in last 100: 88.31
INFO: Reward at episode 1000 is 124.0, avg in last 100: 88.3
INFO: Reward at episode 1100 is 154.0, avg in last 100: 85.62
INFO: Reward at episode 1200 is 86.0, avg in last 100: 85.32
INFO: Reward at episode 1300 is 91.0, avg in last 100: 81.48
INFO: Reward at episode 1400 is 114.0, avg in last 100: 85

In [29]:
x = np.array(range(16)).reshape(4,4)
length = x.shape[0]
batch_size = 3
idxs = np.random.choice(length, batch_size, replace=False)
print("x:",x)
print("idxs:",idxs)
print("x[idxs]:",x[idxs])

x: [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
idxs: [2 3 1]
x[idxs]: [[ 8  9 10 11]
 [12 13 14 15]
 [ 4  5  6  7]]


In [32]:
[a,b,c,d] = x[1]
print(a,b,c,d)

4 5 6 7
